In [1]:
import tensorflow.keras as keras
from ipynb.fs.full.preprocess import generate_training_sequences, SEQUENCE_LENGTH

In [2]:
OUTPUT_UNITS = 38 # Must be the same size as vocabulary size
NUM_UNITS = [256] # num_units = # neurons in internal layers of network
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
EPOCHS = 5 #50 based on previous work
BATCH_SIZE = 64
SAVE_MODEL_PATH = "model.h5"


def build_model(output_units, num_units, loss, learning_rate):
    
    # Create model architecture
    
    # First value None enables us to have as many timestamps as we want; so we can generate melodies of whatever length
    input_layer = keras.layers.Input(shape=(None, output_units))
    # This syntax draws an arrow from input to the LSTM
    x = keras.layers.LSTM(num_units[0])(input_layer)
    x = keras.layers.Dropout(0.2)(x)
    
    # Could add more but we are dealing with a simple model?
    output = keras.layers.Dense(output_units, activation="softmax")(x)
    
    model = keras.Model(input_layer, output)
    
    # Compile model
    model.compile(
        loss=loss, 
        optimizer=keras.optimizers.Adam(lr=learning_rate), 
        metrics=["accuracy"]
    )
    
    model.summary()
    
    return model

In [3]:
def train(output_units=OUTPUT_UNITS, num_units=NUM_UNITS, loss=LOSS, learning_rate=LEARNING_RATE):
    
    # Generate the training sequences
    inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)
    
    # Build the network
    model = build_model(output_units, num_units, loss, learning_rate) 
    
    # Train the model
    model.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)
    
    """
    You will notice that there is no train-test split. In our case, we are not really interested in how the model behaves
    on unseen data. All we want is to get something that works.
    """
    
    # Save the model
    model.save(SAVE_MODEL_PATH)
    

In [4]:
train()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 38)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               302080    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 38)                9766      
Total params: 311,846
Trainable params: 311,846
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
5660/5660 [==============================] - 1746s 308ms/step - loss: 0.6878 - accuracy: 0.7881
Epoch 2/5
5660/5660 [==============================] - 1773s 313ms/step - loss: 0.5816 - accuracy: 0.8118
Epoch 3/5
5660/5660 [====================